In [1]:
#空白联通区域的个数，面积

#存在单连通


# 【连通区域面积小于4】
# 【连通区域面积=4】【坐标】 【坐标】【坐标】【坐标】                                         [水平相对位置][旋转] = 7* 10 * 4 
# 【连通区域面积=8】 找单连通起点【坐标】 【坐标】【坐标】【坐标】【坐标】 【坐标】【坐标】【坐标】        [水平相对位置][旋转] = 7* 10 * 4 
# 【连通区域面积=12】【起点1或2或3】【不是双路区域的联通方向3*3*3】[与边界距离][不是数量最少的形状 随机]     [水平相对位置][旋转] = 7* 10 * 4 

In [2]:
import numpy as np
import cv2
import random
import time
from pynput.keyboard import Key, Controller
import os

import torch
import torch.nn as nn

from collections import deque

In [6]:
pieces = [
        [[1, 1],
         [1, 1]],

        [[0, 2, 0],
         [2, 2, 2]],

        [[0, 3, 3],
         [3, 3, 0]],

        [[4, 4, 0],
         [0, 4, 4]],

        [[5, 5, 5, 5]],

        [[0, 0, 6],
         [6, 6, 6]],

        [[7, 0, 0],
         [7, 7, 7]]
    ]
indexes = [1,2,3,4,5,6,7]
number_of_indexes = [5,5,5,5,5,5,5]

board_width = 10
board_height = 14
total_numbers = 35
block_length = 50

all_x_list = [0,1,2,3,4,5,6]
all_y_list = [0,1,2,3,4,5,6,7,8,9,10]
all_index_list = [0,1,2,3,4,5,6]
all_rotate_list = [0,1,2,3]

directory = r'G:\5Python\savingTetrisState\initpos'
os.chdir(directory)
count_img = [0]

def scaleSize(input_state):
    uint8array = np.array(input_state,dtype = np.uint8)
    realSize_uint8array = [[[0,0,0] for _ in range(board_width * block_length)] for _ in range(board_height * block_length)]

    for i in range(len(uint8array)):
        for j in range(len(uint8array[i])):
            i1 = i * block_length
            j1 = j * block_length
            for k in range(block_length):
                realSize_uint8array[i1][j1] = uint8array[i][j]
                for l in range(block_length):
                    realSize_uint8array[i1 + l][j1 + k] = uint8array[i][j]

    realSize_uint8array = np.array(realSize_uint8array,dtype = np.uint8)
    return realSize_uint8array

def empty_co(state,i,j,x_y_list = [],count=20):
    count -= 1
    if count != 0:
        
        if j >=0 and j < board_height and i >=0 and i < board_width:
            if state[j][i] == [0,0,0]:
                state[j][i] = [255,255,255] 
                x_y_list.append([i,j])
                _ = empty_co(state,j+1,i,x_y_list,count)
                _ = empty_co(state,j,i-1,x_y_list,count)
                _ = empty_co(state,j,i+1,x_y_list,count)
    return x_y_list

def check_collision(original_state,add_new):
    isCollision = False
    start_x = add_new.x
    start_y = add_new.y
    end_x = start_x + len(add_new.piece[0])
    end_y = start_y + len(add_new.piece)

    if start_x < 0:
        isCollision = True
        add_new.xy_move(move_x = 1,move_y = 0)
    if start_y < 0:
        isCollision = True
        add_new.xy_move(move_x = 0,move_y = 1)
    if end_x >= board_width:
        isCollision = True
        add_new.xy_move(move_x = -1,move_y = 0)
    if end_y >= board_height:
        isCollision = True
        add_new.xy_move(move_x = 0,move_y = -1)
        
    for row_idx in range(len(add_new.piece)):
        current_y = start_y + row_idx
        for col_idx, color in enumerate(add_new.piece[row_idx]):
            current_x = start_x + col_idx
            if color!= [0,0,0] and original_state[current_y][current_x] != [0,0,0]:
                isCollision = True
    if isCollision:
        print('Collide! Collide! Collide!')            
    return isCollision

def pieces_opencvShow(input_source,sleep_time_length):
    window_name = 'Show pieces adding to Board! Press Space Key!'
    save_name = '{}.jpg'.format(count_img[0])
    input_source = scaleSize(input_source)
    cv2.imwrite(save_name, input_source)
    count_img[0] += 1
    cv2.imshow(window_name, input_source)
    cv2.waitKey(0)
    keyboard = Controller()
    time.sleep(sleep_time_length)
    keyboard.press('a')
    keyboard.release('a')  
    cv2.destroyAllWindows()
            
            

            
class Pieces:
    def __init__(self,piece,x,y): 
        self.piece = piece
        self.x = x
        self.y = y
        self.piece_selected()
        self.coloredShape()
        
    def piece_selected(self):
        for m in range(len(self.piece)):
            for n in self.piece[m]:
                if n != 0:
                    self.piece_index = n - 1
        return self.piece_index
                    
    def xy_move(self,move_x = 0,move_y = 0):
        if move_x != 0:
            self.x += move_x
        if move_y != 0:
            self.y += move_y
        
    def rotating_pressed(self):
        r = len(pieces[self.piece_index])
        c = len(pieces[self.piece_index][0])
        rotated_piece = [[0]*r for _ in range(c)]
        for row_element in range(r):
            for col_element in range(c):
                it = pieces[self.piece_index][r-row_element-1][col_element]
                rotated_piece[col_element][row_element] = it
        self.piece = rotated_piece
        self.coloredShape()
        #return rotated_piece
    
    def coloredShape(self):
        colors_shape = []
        #row_length = len(pieces[self.piece_selected])
        for i in range(len(self.piece)):
            #col_length = len(pieces[self.piece_selected][i])
            for j in range(len(self.piece[i])):
                if self.piece[i][j] != 0:
                    colors_shape.append(colors[self.piece[i][j]-1])
                else:
                    colors_shape.append([0,0,0])
        self.color_on_shape = colors_shape

        
        
class Board:
    def __init__(self):        
        self.board_width = 10
        self.board_height = 14 
        self.reset()
        
    def reset(self):
        self.score = 0
        self.board_state = [[[0] * 3 for _ in range(self.board_width)] for _ in range(self.board_height)]
        self.empty_list = []
        return self.state_as_nn_input()
        

    def board_add_pieces(self,input_board_state,adding_piece):
        a = adding_piece
        b = input_board_state
        piece_index = Pieces.piece_selected(adding_piece)
        number_of_indexes[piece_index] = number_of_indexes[piece_index] - 1
        collide_or_not = check_collision(b,a)
        if collide_or_not == False:
            color_shape_index = 0
            for row_length in range(len(a.piece)):
                for col_length in range(len(a.piece[row_length])):
                    b[a.y + row_length][a.x + col_length] = a.color_on_shape[color_shape_index]
                    color_shape_index += 1
            step_reward = self.step()
            #pieces_opencvShow(b,1)
            return step_reward
        else:
            print('Not add due to Collision !')
            return 0

    def state_as_nn_input(self):
        heights = self.board_heights()
        
        #count empty area size by using empty_area function
        state_copy = self.board_state.copy()
        i1=0
        while i1 < board_height:
            j1=0
            while j1 < board_width:
                empity_coordinates = empty_co(state_copy,i1,j1,[])
                if empity_coordinates:
                    self.empty_list.append(empity_coordinates)
                j1+=1
            i1+=1
            
        empties = self.empty_list
        
    def board_heights(self):
        highests = []
        for i in range(len(self.board_state[0])):
            length_j = 0
            for j in range(len(self.board_state)):
                if self.board_state[j][i] != [0,0,0] and self.board_state[j][i] != [255,255,255]:
                    length_j = j
            highests.append(length_j)
        return highests
            
    def step(self):
        counter = max_i = max_j = 0
        for j,j_element in enumerate(self.board_state):
            for i,i_element in enumerate(self.board_state[j]):
                if i_element != [0,0,0] and i_element != [255,255,255]:
                    counter+=1
                    max_i = i
                    max_j = j
        bounding_box_size = max_i*max_j
        counter = counter *10
        if bounding_box_size != 0:
            ratio = counter / bounding_box_size
        else:
            return 0
        reward = 0
        if ratio > 0 and ratio < 6.5:
            reward = -1 + counter * 0.03
        elif ratio >= 6.5 and ratio < 8.5:
            reward = (ratio - 6.6)*(ratio - 6.6) + (counter - 5) * 0.035
        elif ratio >= 8.5 and ratio <9.5:
            reward = 4 + counter * 0.04
        else:
            reward = max(5/(1-ratio),15) + counter * 0.03
        return reward
    
    

        
#random pieces parameters
adding_pieces_indexes = [random.choice(all_index_list) for _ in range(total_numbers)]
random_x = [random.choice(all_x_list) for _ in range(total_numbers)]
random_y = [random.choice(all_y_list) for _ in range(total_numbers)]
random_rotate = [random.choice(all_rotate_list) for _ in range(total_numbers)]

colors = [[np.random.choice(range(256)).tolist(),np.random.choice(range(256)).tolist(),np.random.choice(range(256)).tolist()] for _ in range(7)]

#board been created
real_board = Board()

#pieces been created
thirtyFive_pieces = []
for _ in range(len(adding_pieces_indexes)):
    thirtyFive_pieces.append(Pieces(pieces[adding_pieces_indexes[_]],random_x[_],random_y[_]))
    Iscollided = check_collision(real_board.board_state,thirtyFive_pieces[_])
    if Iscollided == True:
        new_counter = 0
        while(new_counter <4):
            thirtyFive_pieces[_].rotating_pressed()
            new_counter += 1
            Iscollided = check_collision(real_board.board_state,thirtyFive_pieces[_])
            if Iscollided == False:
                real_board.board_add_pieces(real_board.board_state,thirtyFive_pieces[_])#OPERATIONS
                print('Piece added:',thirtyFive_pieces[_].piece_index)
            if new_counter ==4:
                print('Piece not added:',thirtyFive_pieces[_].piece_index)
    else:
        real_board.board_add_pieces(real_board.board_state,thirtyFive_pieces[_])#OPERATIONS
        print('Piece added:',thirtyFive_pieces[_].piece_index)

print(real_board.board_state)



r = real_board.board_heights()
print('Board heights:',r)
#pieces_opencvShow(state_copy,10)
print(number_of_indexes)




class DQN(nn.Module):
    def __init__(self):
        super(DQN,self).__init__()
        
        self.conv1 = nn.Sequential(nn.Linear(7 + 1+1 + 20+20), nn.ReLU(inplace = True))
        self.conv2 = nn.Sequential(nn.Linear(512,512), nn.ReLU(inplace = True))
        self.conv3 = nn.Sequential(nn.Linear(512,1))
        
        self._create_weights()
        
    def _create_weights(self):
        for w in self.modules():
            if isinsatnce(w, nn.Linear):
                nn.init.xavier_uniform_(w.weight)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, t):
        t = self.conv1(t)
        t = self.conv2(t)
        t = self.conv3(t)
        return t
    
    
    
epsilon = 0.9

def all_next_board_state(origin_state,shape_indexes,x,y):
    states = {}
    if shape_indexes == 0:
        rot = 1
    elif shape_indexes == 2 or shape_indexes == 3 or shape_indexes == 4:
        rot = 2
    else:
        rot = 4
    
    for i in range(len(shape_indexes)):
        valid_xs = board_width - len(pieces[i][0])
        for x in range(valid_xs + 1):
            for r in range(rot):
                real_piece = Pieces(pieces[shape_indexes],x,y)
                state = board_add_pieces(origin_state,real_piece,real_piece)
                states[(i,x,r)] = self.get_state_properties(state)
                real_piece = rotating_pressed()
                
    return states
    

def train(opt):
    if torch.cude.is_available():
        torch.cuda.munial_seed(123)
    else:
        torch.manual_seed(123)
    
    env = Board()
    model = DQN()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    criterion = nn.MSELoss()
    
    if torch.cuda.is_available():
        model.cude()
        state = state.cuda()
        
    replay_memory = deque(maxlen = 30000)
    epoch = 0
    while epoch < 300:
        next_steps = real_board.board_state.all_next_board_state(real_board.board_state,remained_piece_indexes,start_at_x,start_at_y)
        
        u = random()
        random_action = u <=epsilon
        next_actions, next_states = zip(*next_steps.items())
        next_states = torch.stack(real_board.board_state)
        if torch.cuda.is_available():
            next_states = next_states.cuda()
            
        model.eval()
        with torch.no_grad():
            predictions = model(next_states)[:,0]
            
        model.train()
        if random_action:
            index = randint(0,7*4*10*14 - 1)#7种4旋转x、y坐标
        else:
            index = torch.argmax(predictions).item()
            
        next_state = next_states[index, :]
        action = next_actions[index]
        
        reward = real_board.board_add_pieces(real_board.board_state)
        
        if torch.cuda.is_available():
            next_state = next_state.cuda()
        replay_memory.append([state, reward, next_state])
        
        if len(pieces_list) == 0:
            final_score = real_board.score
            final_empty = real_board.empty_list
            state = env.reset()
            if torch.cuda.is_available():
                state = env.reset()
        else:
            state = next_state
            continue
        if len(replay_memory) < opt.replay_memory_size / 10:
            continue
        
        epoch += 1
        batch = smaple(relay_memory, min(len(replay_memory), batch_size))
        state_batch, reward_batch, next_state_batch, done_batch = zip(*batch)
        state_batch = torch.stack(tuple(state for state in state_batch))
        reward_batch = torch.from_numpy(np.array(reward_batch,dtype = np.float32)[:,None])
        next_state_batch = torch.stack(tuple(statefor state in next_state_batch))
        
        if torch.cuda.is_available():
            state_batch = state_batch.cuda()
            reward_batch = reward_batch.cuda()
            next_state_batch = next_state_batch.cuda()
            
        Q_values = model(state_batch)
        model.evel()
        with torch.no_grad():
            next_prediction_batch = model(next_state_batch)
        model.train()
        
        y_batch = torch.cat()

Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 6
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 3
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 6
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 5
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 1
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Collide!
Piece not added: 3
Collide! Collide! Collide!
Collide! Collide! Collide!
Collide! Collide! Coll